In [ ]:
# --- Cell 1 ---
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="vpython")

In [ ]:
# --- Cell 2 ---
tf = 8.              # total time
dt = 0.01            # simulation time step
S = round(tf/dt) + 1 # number of steps
print(f"S = {S}")

# Parameters
g = 9.81 # gravity (m/s^2)
m = 1.   # mass (m)
J = 0.05 # inertia (kg*m^2)
kx = 0.3
ky = 0.3
k_theta = 0.02

# Limits of thrust (T) and torque (τ)
Tmin = 1/2 * m * g
Tmax = 3/2 * m * g
tau_min = -1/2
tau_max = 1/2

# **1. Modeling**

1. Write down a function in Python called `dynamics()` that takes as input the state $x \in \mathbb{R}^{4 \times 1}$ and controls $u \in \mathbb{R}^{2 \times 1}$ and returns $\dot{x} \in \mathbb{R}^{4 \times 1}.$

In [ ]:
# --- Cell 3 ---
def dynamics(x, u):
    x, y, theta, x_dot, y_dot, theta_dot = x[0,0], x[1,0], x[2,0], x[3,0], x[4,0], x[5,0]
    T, tau = u[0,0], u[1,0]
    vx, vy = x_dot, y_dot
    
    fx = -kx * vx * np.abs(vx)
    fy = -ky * vy * np.abs(vy)
    f_theta = -k_theta * theta_dot
    
    x_ddot = 1/m * (-np.sin(theta) * (T + fy) + np.cos(theta) * fx)
    y_ddot = 1/m * (np.cos(theta) * (T + fy) + np.sin(theta) * fx) - g
    theta_ddot = (tau + f_theta) / J
    
    return np.array([[x_dot, y_dot, theta_dot, x_ddot, y_ddot, theta_ddot]]).T

2. Create a simple visualization of the system (e.g. by using the matplotlib library); you are
free to use any library for the visualization.

In [ ]:
# --- Cell 4 ---
def simulation(xs, x0, x_ref):
    states = np.hstack(xs)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_aspect('equal')
    ax.grid(True)
    ax.set_title('Modeling of Planar Quadrotor')
    
    L = 0.25
    quadrotor_L,  = ax.plot([], [], 'o-', lw=1, color='black')
    init_point,   = ax.plot([], [], 'bo', markersize=5)
    target_point, = ax.plot([], [], 'ro', markersize=5)
    
    def init():
        quadrotor_L.set_data([], [])
        init_point.set_data([x0[0,0]], [x0[1,0]])
        target_point.set_data([x_ref[0,0]], [x_ref[1,0]])
        
        return quadrotor_L, init_point, target_point
    
    def animate(i):
        # Get current state
        x = states[0, i]
        y = states[1, i]
        theta = states[2, i]
        
        # Left side
        xl = x - L * np.cos(theta)
        yl = y - L * np.sin(theta)
        # Right side
        xr = x + L * np.cos(theta)
        yr = y + L * np.sin(theta)
        
        # Update quadrotor's position
        quadrotor_L.set_data([xl, xr], [yl, yr])
        
        return quadrotor_L,
    
    # Create animation
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=S, interval=20, blit=True)
    # anim.save("modeling.mp4", writer='ffmpeg', fps=30, dpi=100)
    
    plt.close()
    
    return HTML(anim.to_jshtml())

3. Discretize the system using **Runge-Kutta 4th Order integration**.

In [ ]:
# --- Cell 5 ---
# Runge-Kutta 4th Order integration
def rk4(x, u, dt):
    f1 = dynamics(x, u)
    f2 = dynamics(x + f1 * dt/2, u)
    f3 = dynamics(x + f2 * dt/2, u)
    f4 = dynamics(x + f3 * dt, u)
    
    return x + dt/6 * (f1 + 2*f2 + 2*f3 + f4)

4. Make sure that the integration works and is correct.

In [ ]:
# --- Cell 6 ---
def pd_controller(x, x_ref, kpx=6., kdx=3., kpy=40., kdy=8., kpt=60., kdt=5.):
    x, y, theta, vx, vy, v_theta = x[0,0], x[1,0], x[2,0], x[3,0], x[4,0], x[5,0]
    x_ref, desired_y, desired_theta, desired_vx, desired_vy, desired_v_theta = x_ref[0,0], x_ref[1,0], x_ref[2,0], x_ref[3,0], x_ref[4,0], x_ref[5,0]
    
    fx = -kx * vx * np.abs(vx)
    fy = -ky * vy * np.abs(vy)
    f_theta = -k_theta * v_theta

    # x controller
    x_error = x_ref - x
    vx_error = desired_vx - vx
    desired_ax = kpx * x_error + kdx * vx_error

    # y controller
    y_error = desired_y - y
    vy_error = desired_vy - vy
    desired_ay = kpy * y_error + kdy * vy_error

    # θ controller
    desired_theta = - desired_ax / g # approximation of θ
    theta_error = desired_theta - theta
    v_theta_error = desired_v_theta - v_theta
    desired_a_theta = kpt * theta_error + kdt * v_theta_error

    # Control Signals
    T = (-np.cos(theta) * fy - np.sin(theta) * fx + m * desired_ay + m * g) / np.cos(theta)
    tau = f_theta + J * desired_a_theta
    
    # Clamp of u = [T, τ]^T
    T = max(Tmin, min(T, Tmax))
    tau = max(tau_min, min(tau, tau_max))
    
    return np.array([[T, tau]]).T

In [ ]:
# --- Cell 7 ---
# Initial state and control
x0 = np.array([[0., 0., 0., 0., 0., 0.]]).T
u0 = np.array([[0., 0.]]).T

# Reference state
x_ref = np.array([[-0.5, 0.5, 0., 0., 0., 0.]]).T
 
x = np.copy(x0)
xs = [np.copy(x)]
us = []

for i in range(S):
    # Update control input
    u = pd_controller(x, x_ref)
    us.append(np.copy(u))

    # Update state
    x = rk4(x, u, dt)
    xs.append(np.copy(x))

print("Program found the solution!")
# print("Wait for simulation...")
# simulation(xs, x0, x_ref)

In [ ]:
# --- Cell 8 ---
# Let's plot it!
xs = np.array(xs)
us = np.array(us)

fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(131)
ax1.plot([n*dt for n in range(len(xs))], xs[:,0], label=r'x')
ax1.plot([n*dt for n in range(len(xs))], xs[:,1], label=r'y')
ax1.plot([n*dt for n in range(len(xs))], xs[:,2], label=r'θ')
ax1.legend()

ax2 = fig.add_subplot(132)
ax2.plot([n*dt for n in range(len(xs))], xs[:,3], label=r'$\dot{x}$')
ax2.plot([n*dt for n in range(len(xs))], xs[:,4], label=r'$\dot{y}$')
ax2.plot([n*dt for n in range(len(xs))], xs[:,5], label=r'$\dot{θ}$')
ax2.legend()

ax3 = fig.add_subplot(133)
ax3.plot([n*dt for n in range(len(us))], us[:,0], label=r'T')
ax3.plot([n*dt for n in range(len(us))], us[:,1], label=r'τ')
ax3.legend()

plt.show()